In [1]:
from db import Eleicao, Estado, Municipio, Zona, Secao, Log, MunicipioZona
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from datetime import datetime as dt
from ipywidgets import widgets
from ipywidgets import interact, interact_manual, interactive_output, VBox, HBox, Accordion, Box, Layout

sns.set_palette("bright")

# Plot Function
def plot_graph(secao1=1,secao2=1,secao3=1,secao4=1,eleicao1=1,eleicao2=1,eleicao3=1,eleicao4=1,):
    if(not secao1 or not secao2 or not secao3 or not secao4):
        return 
    else:
        plt.figure(dpi=120)
        sns.set_style('whitegrid')
        
        #liberar grafico
       
#Functions

def get_dataframes(secao, eleicao):  
        mes = 10
        if(eleicao == 1):
            dia = 2
        else:
            dia = 30

        query = Log.select().where((Log.secao == secao) & (Log.datetime.day == dia) & (Log.datetime.month == mes) )
        df = pd.DataFrame(list(query.dicts()))

        #quantidade de votos
        votos = df[df.descricao == 'O voto do eleitor foi computado']
        # votos_d.value = str(votos.shape[0])
        
        #comeco e fim das atividades
        inicio = df.iloc[0]
        fim = df.iloc[-1]

        # inicio_d.value = str(inicio['datetime'].time())
        # fim_d.value = str(fim['datetime'].time())

        #primeiro e ultimo votos
        primeiro = votos.iloc[0]
        ultimo = votos.iloc[-1]

        # primeiro_d.value = str(primeiro['datetime'].time())
        # ultimo_d.value = str(ultimo['datetime'].time())

        #média de votos por hora
        #horas
        horas = pd.to_datetime(df.datetime).dt.hour.unique()

        #resultados
        result = pd.DataFrame(data=horas, columns=['Horas'])


        # Dividir por hora
        logs_por_hora = [g.reset_index(drop=True) for i,g in df.groupby(pd.to_datetime(df.datetime).dt.hour)]

        cont = 0
        votos = []

        for dy in logs_por_hora:
            voto = dy[dy.descricao == 'O voto do eleitor foi computado'].shape[0]
            votos.append(voto)
            cont += 1

        result['Votos'] = votos
        result['Porcentagem'] = result['Votos'] / result['Votos'].sum()

        votos_hora = result[result['Votos'] != 0]

        # media_votos_hora_d.value = str(round(votos_hora['Votos'].mean(),2))

        #tempo médio de votação
        def format(tdelta):
            horas, rem =  divmod(tdelta.seconds, 3600)
            minutos, segundos = divmod(rem, 60)
            time = '{:02d}:{:02d}:{:02d}'.format(horas,minutos, segundos)
            return time
        

        comeco = df.reset_index().index[df.descricao == 'Título digitado pelo mesário'].tolist()
        fim = df.reset_index().index[df.descricao == 'O voto do eleitor foi computado'].tolist()
        comeco.reverse()
        fim.reverse()

        xs = []
        temp = []
        for b in fim:
            for a in comeco:
                if(a < b):
                    xs.append(a)
                    temp.append((a,b))
                    break

        temp.reverse()
        comeco.reverse()
        fim.reverse()
        xs.reverse()
        temp2 = [c for c in comeco if c not in xs]


        tempos = []
        eleitores = [df.iloc[temp[d][0]:temp[d][1]] for d in range(0, len(temp)) ]
        for eleitor in eleitores:
            inicio = eleitor.iloc[0]['datetime']
            fim = eleitor.iloc[-1]['datetime']
            diff = (fim - inicio)
            tempos.append(diff)
            # tempos.append(pd.to_datetime().dt.time)

        temp_votacao  = pd.DataFrame(tempos, columns=['Tempo em Votação'])
        # tempo_medio_vota_d.value = format(temp_votacao['Tempo em Votação'].mean())

        #tempo médio na urna
        
        comeco = df.reset_index().index[df.descricao == 'Eleitor foi habilitado'].tolist()
        fim = df.reset_index().index[df.descricao == 'O voto do eleitor foi computado'].tolist()
        comeco.reverse()
        fim.reverse()

        xs = []
        temp = []
        for b in fim:
            for a in comeco:
                if(a < b):
                    xs.append(a)
                    temp.append((a,b))
                    break

        temp.reverse()
        comeco.reverse()
        fim.reverse()
        xs.reverse()


        tempos = []
        eleitores = [df.iloc[temp[d][0]:temp[d][1]] for d in range(0, len(temp)) ]
        for eleitor in eleitores:
            inicio = eleitor.iloc[0]['datetime']
            fim = eleitor.iloc[-1]['datetime']
            diff = (fim - inicio)
            tempos.append(diff)
            

        temp_urna  = pd.DataFrame(tempos, columns=['Tempo na Urna'])

        # tempo_medio_na_urna_d.value = format(temp_urna['Tempo na Urna'].mean())

        #dificuldade na votação

        comeco = df.reset_index().index[df.descricao == 'Eleitor foi habilitado'].tolist()
        fim = df.reset_index().index[df.descricao == 'O voto do eleitor foi computado'].tolist()

        comeco.reverse()
        fim.reverse()

        xs = []
        temp = []
        for n in fim:
            for m in comeco:
                if(m < n):
                    xs.append(m)
                    temp.append((m,n))
                    break

        temp.reverse()
        comeco.reverse()
        fim.reverse()
        xs.reverse()

        erros = []
        eleitores = [df.iloc[temp[x][0]:temp[x][1]] for x in range(0, len(temp)) ]
        for eleitor in eleitores:
            erros_durante_votacao = eleitor[eleitor.descricao == 'Tecla indevida pressionada'].shape[0]
            erros.append(erros_durante_votacao)

        votacao = pd.DataFrame(erros, columns=['Erros na Votação'])

        #dificuldade no reconhecimento da digital
        comeco = df.reset_index().index[df.descricao == 'Título digitado pelo mesário'].tolist()
        fim = df.reset_index().index[df.descricao == 'O voto do eleitor foi computado'].tolist()
        comeco.reverse()
        fim.reverse()

        xs = []
        temp = []
        for n in fim:
            for m in comeco:
                if(m < n):
                    xs.append(m)
                    temp.append((m,n))
                    break

        temp.reverse()
        comeco.reverse()
        fim.reverse()
        xs.reverse()


        erros_digitais = []
        eleitores = [df.iloc[temp[x][0]:temp[x][1]] for x in range(0, len(temp)) ]
        for eleitor in eleitores:
            erro_digital = eleitor[eleitor['descricao'].str.contains('Digital capturada não corresponde a digital do eleitor')].shape[0]
            erros_digitais.append(erro_digital)

        digital  = pd.DataFrame(erros_digitais, columns=['Erros na Digital'])
        
        #agregando em um único dataset

        database = temp_votacao.join(temp_urna).join(digital).join(votacao)

        return []

def observe_eleicao1(_):
    if(eleicao_d1.value >= 1):
        query = Estado.select().where(Estado.eleicao == eleicao_d1.value)
        b = list(query.dicts())
        def get_list_estado(esta):
            return (esta['nome'], esta['id'])
        estados = [('Selecione o Estado...', 0)] +list(map(get_list_estado, b))
        estado_d1.options = estados
        estado_d1.disabled = False

def observe_estado1(_):
    if(estado_d1.value >= 1):
        query = Municipio.select().where(Municipio.estado == estado_d1.value)
        c = list(query.dicts())
        def get_list_municipio(muni):
            return (muni['nome'], muni['id'])
        municipios = [('Selecione o Municipio...', 0)] + list(map(get_list_municipio, c))
        municipio_d1.options = municipios
        municipio_d1.disabled = False
    
def observe_municipio1(_):
    if(municipio_d1.value >= 1):
        query = Zona.select().join(MunicipioZona).where(MunicipioZona.municipio == municipio_d1.value)
        d = list(query.dicts())
        def get_list_zona(zon):
            return (zon['nome'], zon['id'])
        zonas = [('Selecione a Zona...', 0)] + list(map(get_list_zona, d))
        zona_d1.options = zonas
        zona_d1.disabled = False

def observe_zona1(_):
    if(zona_d1.value >= 1 & municipio_d1.value >= 1):
        query = Secao.select().where((Secao.municipio == municipio_d1.value) & (Secao.zona == zona_d1.value))
        e = list(query.dicts())
        def get_list_secao(sec):
            return (sec['nome'], sec['id'])
        secoes = [('Selecione a Seção...', 0)] + list(map(get_list_secao, e))
        secao_d1.options = secoes
        secao_d1.disabled = False

#Funções 2
def observe_eleicao2(_):
    if(eleicao_d2.value >= 1):
        query = Estado.select().where(Estado.eleicao == eleicao_d2.value)
        b = list(query.dicts())
        def get_list_estado(esta):
            return (esta['nome'], esta['id'])
        estados = [('Selecione o Estado...', 0)] +list(map(get_list_estado, b))
        estado_d2.options = estados
        estado_d2.disabled = False

def observe_estado2(_):
    if(estado_d2.value >= 1):
        query = Municipio.select().where(Municipio.estado == estado_d2.value)
        c = list(query.dicts())
        def get_list_municipio(muni):
            return (muni['nome'], muni['id'])
        municipios = [('Selecione o Municipio...', 0)] + list(map(get_list_municipio, c))
        municipio_d2.options = municipios
        municipio_d2.disabled = False
    
def observe_municipio2(_):
    if(municipio_d2.value >= 1):
        query = Zona.select().join(MunicipioZona).where(MunicipioZona.municipio == municipio_d2.value)
        d = list(query.dicts())
        def get_list_zona(zon):
            return (zon['nome'], zon['id'])
        zonas = [('Selecione a Zona...', 0)] + list(map(get_list_zona, d))
        zona_d2.options = zonas
        zona_d2.disabled = False

def observe_zona2(_):
    if(zona_d2.value >= 1 & municipio_d2.value >= 1):
        query = Secao.select().where((Secao.municipio == municipio_d2.value) & (Secao.zona == zona_d2.value))
        e = list(query.dicts())
        def get_list_secao(sec):
            return (sec['nome'], sec['id'])
        secoes = [('Selecione a Seção...', 0)] + list(map(get_list_secao, e))
        secao_d2.options = secoes
        secao_d2.disabled = False

#Funções 3
def observe_eleicao3(_):
    if(eleicao_d3.value >= 1):
        query = Estado.select().where(Estado.eleicao == eleicao_d3.value)
        b = list(query.dicts())
        def get_list_estado(esta):
            return (esta['nome'], esta['id'])
        estados = [('Selecione o Estado...', 0)] +list(map(get_list_estado, b))
        estado_d3.options = estados
        estado_d3.disabled = False

def observe_estado3(_):
    if(estado_d3.value >= 1):
        query = Municipio.select().where(Municipio.estado == estado_d3.value)
        c = list(query.dicts())
        def get_list_municipio(muni):
            return (muni['nome'], muni['id'])
        municipios = [('Selecione o Municipio...', 0)] + list(map(get_list_municipio, c))
        municipio_d3.options = municipios
        municipio_d3.disabled = False
    
def observe_municipio3(_):
    if(municipio_d3.value >= 1):
        query = Zona.select().join(MunicipioZona).where(MunicipioZona.municipio == municipio_d3.value)
        d = list(query.dicts())
        def get_list_zona(zon):
            return (zon['nome'], zon['id'])
        zonas = [('Selecione a Zona...', 0)] + list(map(get_list_zona, d))
        zona_d3.options = zonas
        zona_d3.disabled = False

def observe_zona3(_):
    if(zona_d3.value >= 1 & municipio_d3.value >= 1):
        query = Secao.select().where((Secao.municipio == municipio_d3.value) & (Secao.zona == zona_d3.value))
        e = list(query.dicts())
        def get_list_secao(sec):
            return (sec['nome'], sec['id'])
        secoes = [('Selecione a Seção...', 0)] + list(map(get_list_secao, e))
        secao_d3.options = secoes
        secao_d3.disabled = False

#Funções 4
def observe_eleicao4(_):
    if(eleicao_d4.value >= 1):
        query = Estado.select().where(Estado.eleicao == eleicao_d4.value)
        b = list(query.dicts())
        def get_list_estado(esta):
            return (esta['nome'], esta['id'])
        estados = [('Selecione o Estado...', 0)] +list(map(get_list_estado, b))
        estado_d4.options = estados
        estado_d4.disabled = False

def observe_estado4(_):
    if(estado_d4.value >= 1):
        query = Municipio.select().where(Municipio.estado == estado_d4.value)
        c = list(query.dicts())
        def get_list_municipio(muni):
            return (muni['nome'], muni['id'])
        municipios = [('Selecione o Municipio...', 0)] + list(map(get_list_municipio, c))
        municipio_d4.options = municipios
        municipio_d4.disabled = False
    
def observe_municipio4(_):
    if(municipio_d4.value >= 1):
        query = Zona.select().join(MunicipioZona).where(MunicipioZona.municipio == municipio_d4.value)
        d = list(query.dicts())
        def get_list_zona(zon):
            return (zon['nome'], zon['id'])
        zonas = [('Selecione a Zona...', 0)] + list(map(get_list_zona, d))
        zona_d4.options = zonas
        zona_d4.disabled = False

def observe_zona4(_):
    if(zona_d4.value >= 1 & municipio_d4.value >= 1):
        query = Secao.select().where((Secao.municipio == municipio_d4.value) & (Secao.zona == zona_d4.value))
        e = list(query.dicts())
        def get_list_secao(sec):
            return (sec['nome'], sec['id'])
        secoes = [('Selecione a Seção...', 0)] + list(map(get_list_secao, e))
        secao_d4.options = secoes
        secao_d4.disabled = False

# def observe_secao(_):
#     if(secao_d1.value >= 0):
#         tipo_d1.disabled = False

# def observe_tipo(_):
#     x_d1.disabled = False
#     if(tipo_d.value == 'Histograma' or tipo_d.value == 'Calor'):
#         x_d1.options = opcoes2
#         y_d1.disabled = True
#     else:
#         x_d1.options = opcoes
#         y_d1.disabled = False


#Default
query = Eleicao.select()
a = list(query.dicts())
def get_list_eleicao(eleicao):
    return ('{} - {}º turno'.format(eleicao['ano'],eleicao['turno']), eleicao['id'])
eleicoes = [('Selecione a Eleição...', 0)] + list(map(get_list_eleicao, a))
estados = [('Selecione o Estado...', 0)]
municipios = [('Selecione o Municipio...', 0)]
zonas = [('Selecione a Zona...', 0)]
secoes = [('Selecione a Seção...', 0)]
opcoes = ['Tempo em Votação', 'Tempo na Urna', 'Erros na Digital', 'Erros na Votação']
opcoes2 = ['Votos por Hora', 'Porcentagem por Hora',  'Tempo em Votação', 'Tempo na Urna', 'Erros na Digital', 'Erros na Votação']
tipos = ['Histograma', 'Dispersão', 'Calor']

style = {'description_width': 'initial'}

#Entradas 1
eleicao_d1 = widgets.Dropdown(
    options = eleicoes,
    value = eleicoes[0][1],
    description = 'Eleição'
)
estado_d1 = widgets.Dropdown(
    options = estados,
    value = estados[0][1],
    description = "Estado",
    disabled = True
)

municipio_d1 = widgets.Dropdown(
    options = municipios,
    value = municipios[0][1],
    description = "Municipio",
    disabled = True
)
zona_d1 = widgets.Dropdown(
    options = zonas,
    value = zonas[0][1],
    description = "Zona",
    disabled = True
)
secao_d1 = widgets.Dropdown(
    options = secoes,
    value = secoes[0][1],
    description = "Seção",
    disabled = True
)
# tipo_d1 = widgets.Dropdown(
#     options = tipos,
#     value = tipos[0],
#     description = "Tipo",
#     disabled = True
# )
# x_d1 = widgets.Dropdown(
#     options = opcoes,
#     value = opcoes[0],
#     description = "X",
#     disabled = True
# )
# y_d1 = widgets.Dropdown(
#     options = opcoes,
#     value = opcoes[1],
#     description = "Y",
#     disabled = True
# )

#Entradas 2
eleicao_d2 = widgets.Dropdown(
    options = eleicoes,
    value = eleicoes[0][1],
    description = 'Eleição'
)
estado_d2 = widgets.Dropdown(
    options = estados,
    value = estados[0][1],
    description = "Estado",
    disabled = True
)

municipio_d2 = widgets.Dropdown(
    options = municipios,
    value = municipios[0][1],
    description = "Municipio",
    disabled = True
)
zona_d2 = widgets.Dropdown(
    options = zonas,
    value = zonas[0][1],
    description = "Zona",
    disabled = True
)
secao_d2 = widgets.Dropdown(
    options = secoes,
    value = secoes[0][1],
    description = "Seção",
    disabled = True
)
# tipo_d2 = widgets.Dropdown(
#     options = tipos,
#     value = tipos[0],
#     description = "Tipo",
#     disabled = True
# )
# x_d2 = widgets.Dropdown(
#     options = opcoes,
#     value = opcoes[0],
#     description = "X",
#     disabled = True
# )
# y_d2 = widgets.Dropdown(
#     options = opcoes,
#     value = opcoes[1],
#     description = "Y",
#     disabled = True
# )

#Entradas 3
eleicao_d3 = widgets.Dropdown(
    options = eleicoes,
    value = eleicoes[0][1],
    description = 'Eleição'
)
estado_d3 = widgets.Dropdown(
    options = estados,
    value = estados[0][1],
    description = "Estado",
    disabled = True
)

municipio_d3 = widgets.Dropdown(
    options = municipios,
    value = municipios[0][1],
    description = "Municipio",
    disabled = True
)
zona_d3 = widgets.Dropdown(
    options = zonas,
    value = zonas[0][1],
    description = "Zona",
    disabled = True
)
secao_d3 = widgets.Dropdown(
    options = secoes,
    value = secoes[0][1],
    description = "Seção",
    disabled = True
)
# tipo_d3 = widgets.Dropdown(
#     options = tipos,
#     value = tipos[0],
#     description = "Tipo",
#     disabled = True
# )
# x_d3 = widgets.Dropdown(
#     options = opcoes,
#     value = opcoes[0],
#     description = "X",
#     disabled = True
# )
# y_d3 = widgets.Dropdown(
#     options = opcoes,
#     value = opcoes[1],
#     description = "Y",
#     disabled = True
# )

#Entradas 4
eleicao_d4 = widgets.Dropdown(
    options = eleicoes,
    value = eleicoes[0][1],
    description = 'Eleição'
)
estado_d4 = widgets.Dropdown(
    options = estados,
    value = estados[0][1],
    description = "Estado",
    disabled = True
)

municipio_d4 = widgets.Dropdown(
    options = municipios,
    value = municipios[0][1],
    description = "Municipio",
    disabled = True
)
zona_d4 = widgets.Dropdown(
    options = zonas,
    value = zonas[0][1],
    description = "Zona",
    disabled = True
)
secao_d4 = widgets.Dropdown(
    options = secoes,
    value = secoes[0][1],
    description = "Seção",
    disabled = True
)
# tipo_d4 = widgets.Dropdown(
#     options = tipos,
#     value = tipos[0],
#     description = "Tipo",
#     disabled = True
# )
# x_d4 = widgets.Dropdown(
#     options = opcoes,
#     value = opcoes[0],
#     description = "X",
#     disabled = True
# )
# y_d4 = widgets.Dropdown(
#     options = opcoes,
#     value = opcoes[1],
#     description = "Y",
#     disabled = True
# )

# color_yd = widgets.ColorPicker(
#     concise=False,
#     description='Cor Y',
#     value='#b2e061',
#     disabled=False
# )


#Dados
# votos_d = widgets.Text(
#     value = '',
#     description = "Votos Computados",
#     disabled = True,
#     style=style
# )

# inicio_d = widgets.Text(
#     value = '',
#     description = "Inicio das Atividades",
#     disabled = True,
#     style=style
# )

# fim_d = widgets.Text(
#     value = '',
#     description = "Fim das Atividades",
#     disabled = True,
#     style=style
# )

# primeiro_d = widgets.Text(
#     value = '',
#     description = "Primeiro Voto da Seção",
#     disabled = True,
#     style=style
# )

# ultimo_d = widgets.Text(
#     value = '',
#     description = "Último Voto da Seção",
#     disabled = True,
#     style=style
# )

# media_votos_hora_d = widgets.Text(
#     value = '',
#     description = "Média de Votos por Hora",
#     disabled = True,
#     style=style
# )

# tempo_medio_vota_d = widgets.Text(
#     value = '',
#     description = "Tempo Médio de Votação",
#     disabled = True,
#     style=style
# )

# tempo_medio_na_urna_d = widgets.Text(
#     value = '',
#     description = "Tempo Médio na Urna",
#     disabled = True,
#     style=style
# )




#Observe
eleicao_d1.observe(observe_eleicao1)
estado_d1.observe(observe_estado1)
municipio_d1.observe(observe_municipio1)
zona_d1.observe(observe_zona1)

#Observe
eleicao_d2.observe(observe_eleicao2)
estado_d2.observe(observe_estado2)
municipio_d2.observe(observe_municipio2)
zona_d2.observe(observe_zona2)


#Observe
eleicao_d3.observe(observe_eleicao3)
estado_d3.observe(observe_estado3)
municipio_d3.observe(observe_municipio3)
zona_d3.observe(observe_zona3)


#Observe
eleicao_d4.observe(observe_eleicao4)
estado_d4.observe(observe_estado4)
municipio_d4.observe(observe_municipio4)
zona_d4.observe(observe_zona4)



out = interactive_output(plot_graph,{
    "secao1" : secao_d1,
    "secao2" : secao_d2,
    "secao3" : secao_d3,
    "secao4" : secao_d4,
    
    },
       
)

#Layout
ui = HBox([eleicao_d1, estado_d1,municipio_d1, zona_d1, secao_d1], layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid red 2px',
    align_items='center',
    justify_content = 'space-around',
    flex = '3 1 auto',
    width = 'auto'
))
ui2 = HBox([eleicao_d2, estado_d2,municipio_d2, zona_d2, secao_d2], layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid red 2px',
    align_items='center',
    justify_content = 'space-around',
    flex = '3 1 auto',
    width = 'auto'
))
ui3 = HBox([eleicao_d3, estado_d3,municipio_d3, zona_d3, secao_d3], layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid red 2px',
    align_items='center',
    justify_content = 'space-around',
    flex = '3 1 auto',
    width = 'auto'
))
ui4 = HBox([eleicao_d4, estado_d4,municipio_d4, zona_d4, secao_d4], layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid red 2px',
    align_items='center',
    justify_content = 'space-around',
    flex = '3 1 auto',
    width = 'auto'
))

box2 = HBox([out], layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid red 2px',
    align_items='center',
    justify_content = 'space-around',
    flex = '3 1 auto',
    width = 'auto'
))
data = HBox([ui,ui2,ui3,ui4])
box = HBox([data,box2], layout=Layout(
    display='flex',
    flex_flow='row',
    align_items='stretch',
    justify_content = 'space-around', 
))


display(box)

